# First Order Predicate Logic

### Task 0


In [2]:
from operator import add
from LPTester import test_batch

CONSTANT = 'constant'
VARIABLE = 'variable'
FUNCTION = 'function'
ATOM = 'atom'
FORMULA = 'formula'
NEG = 'not'
OR = 'or'
AND = 'and'

In [4]:
def make_const(value):
    return [CONSTANT, value]
def make_var(name):
    return [VARIABLE, name, None]
def make_function_call(function, *args):
    return [FUNCTION, function, list(args)]
def make_atom(predicate, *args):
    return [FORMULA, ATOM, predicate, list(args)]
def make_neg(sentence):
    return [FORMULA, NEG, [sentence]]
def make_and(sentence1, sentence2, *others):
    return [FORMULA, AND, [sentence1, sentence2] + list(others)]
def make_or(sentence1, sentence2, *others):
    return [FORMULA, OR, [sentence1, sentence2] + list(others)]
def replace_args(formula, new_args):
    if formula[0] == FUNCTION:
        return [FUNCTION, formula[1], list(new_args)]
    elif formula[0] == FORMULA:
        if formula[1] == ATOM:
            return [FORMULA, ATOM, formula[2], list(new_args)]
        else:
            return [formula[0], formula[1], list(new_args)]
    return formula

In [5]:
def is_term(f):
    return is_constant(f) or is_variable(f) or is_function_call(f)
def is_constant(f):
    return f[0] == CONSTANT
def is_variable(f):
    return f[0] == VARIABLE
def is_function_call(f):
    return f[0] == FUNCTION
def is_atom(f):
    return f[0] == FORMULA and f[1] == ATOM
def is_sentence(f):
    return f[0] == FORMULA
def has_args(f):
    return is_function_call(f) or is_sentence(f)
def get_value(f):
    if is_constant(f):
        return f[1]
    return None
def get_name(f):
    if is_variable(f):
        return f[1]
    return None
def get_head(f):
    if is_function_call(f):
        return f[1]
    elif f[0] == FORMULA:
        if is_atom(f):
            return f[2]
        else:
            return f[1]
    return None
def get_args(f):
    if is_function_call(f):
        return f[2]
    elif is_atom(f):
        return f[3]
    elif is_sentence(f):
        return f[2]
    return None

test_batch(0, globals())
test_batch(0, globals())

>>> Test batch [0]
Test 0: OK
Test 1: OK
Test 2: OK
Test 3: OK
Test 4: OK
Test 5: OK
Test 6: OK
Test 7: OK
Test 8: OK
Test 9: OK
Test 10: OK
Test 11: OK
Test 12: OK
Test 13: OK
Test 14: OK
Test 15: OK
Test 16: OK
Test 17: OK
Test 18: OK
Test 19: OK
Test 20: OK
Test 21: OK
Test 22: OK
Test 23: OK
Test 24: OK
Test 25: OK
Test 26: OK
Test 27: OK
Test 28: OK
Test 29: OK
Test 30: OK
Test 31: OK
Test 32: OK
>>>  33 / 33 tests successful.
>>> Test batch [0]
Test 0: OK
Test 1: OK
Test 2: OK
Test 3: OK
Test 4: OK
Test 5: OK
Test 6: OK
Test 7: OK
Test 8: OK
Test 9: OK
Test 10: OK
Test 11: OK
Test 12: OK
Test 13: OK
Test 14: OK
Test 15: OK
Test 16: OK
Test 17: OK
Test 18: OK
Test 19: OK
Test 20: OK
Test 21: OK
Test 22: OK
Test 23: OK
Test 24: OK
Test 25: OK
Test 26: OK
Test 27: OK
Test 28: OK
Test 29: OK
Test 30: OK
Test 31: OK
Test 32: OK
>>>  33 / 33 tests successful.


In [6]:
def print_formula(f, return_result = False):
    ret = ""
    if is_term(f):
        if is_constant(f):
            ret += str(get_value(f))
        elif is_variable(f):
            ret += "?" + get_name(f)
        elif is_function_call(f):
            ret += str(get_head(f)) + "[" + "".join([print_formula(arg, True) + "," for arg in get_args(f)])[:-1] + "]"
        else:
            ret += "???"
    elif is_atom(f):
        ret += str(get_head(f)) + "(" + "".join([print_formula(arg, True) + ", " for arg in get_args(f)])[:-2] + ")"
    elif is_sentence(f):
        args = get_args(f)
        if len(args) == 1:
            ret += str(get_head(f)) + print_formula(args[0], True)
        else:
            ret += "(" + str(get_head(f)) + "".join([" " + print_formula(arg, True) for arg in get_args(f)]) + ")"
    else:
        ret += "???"
    if return_result:
        return ret
    print(ret)
    return
formula1 = make_and(
    make_or(make_neg(make_atom("P", make_var("x"))), make_atom("Q", make_var("x"))),
    make_atom("T", make_var("y"), make_function_call(add, make_const(1), make_const(2))))
print_formula(formula1)

(and (or notP(?x) Q(?x)) T(?y, <built-in function add>[1,2]))


## Unification

In [7]:
def substitute(f, substitution):
    if substitution is None:
        return None
    if is_variable(f) and (get_name(f) in substitution):
        return substitute(substitution[get_name(f)], substitution)
    if has_args(f):
        return replace_args(f, [substitute(arg, substitution) for arg in get_args(f)])
    return f

def test_formula(x, copyy = False):
    fun = make_function_call(add, make_const(1), make_const(2))
    return make_and(make_or(make_neg(make_atom("P", make_const(x))), make_atom("Q", make_const(x))), \
                    make_atom("T", fun if copyy else make_var("y"), fun))

test_batch(1, globals())

>>> Test batch [1]
Test 0: OK
Test 1: OK
Test 2: OK
Test 3: OK
Test 4: OK
>>>  5 / 5 tests successful.


### Task 1


In [8]:
def occur_check(v, t, subst):
    if v == t:
        return True
    if get_name(t) in subst:
        return occur_check(v, substitute(t, subst), subst)
    if is_function_call(t):
        for arg in t[2]:
            if occur_check(v, arg, subst):
                return True
    return False

test_batch(2, globals())

>>> Test batch [2]
Test 0: OK
Test 1: OK
Test 2: OK
Test 3: OK
Test 4: OK
Test 5: OK
Test 6: OK
>>>  7 / 7 tests successful.


In [9]:
def unify(f1, f2, subst = None):
    if subst is None:
        subst = {}
    stack = []
    stack.append((f1, f2))
    while len(stack) > 0:
        (f1, f2) = stack.pop()
        
        while get_name(f1) in subst:
            f1 = substitute(f1, subst)
        while get_name(f2) in subst:
            f2 = substitute(f2, subst)
        
        if not f1 == f2:
            if is_variable(f1):
                if occur_check(f1, f2, subst):
                    return False
                else:
                    subst[get_name(f1)] = f2
            elif is_variable(f2):
                if occur_check(f2, f1, subst):
                    return False
                else:
                    subst[get_name(f2)] = f1
            
            elif has_args(f1) and has_args(f2):
                hs = get_args(f1)
                ht = get_args(f2)
                
                if get_head(f1) == get_head(f2) and len(hs) == len(ht):
                    n = len(hs)
                    for _ in range(n):
                        stack.append((hs[_], ht[_]))
                else:
                    return False
            else:
                return False
    return subst
test_batch(3, globals())







>>> Test batch [3]
Test 0: OK, got < False >
Test 1: OK, got < {?x -> A} >
Test 2: OK, got < {?x -> A} >
Test 3: OK, got < {?x -> A} >
Test 4: OK, got < {?x -> A} >
Test 5: OK, got < {?x -> <built-in function add>[?Z,5]} >
Test 6: OK, got < {?z -> C, ?y -> B, ?x -> A} >
Test 7: OK, got < {?y -> 3, ?x -> 2} >
Test 8: OK, got < False >
Test 9: OK, got < False >
Test 10: OK, got < {} >
Test 11: OK, got < {?x -> ?y} >
Test 12: OK, got < {?y -> ?x} >
Test 13: OK, got < {?x -> A} >
Test 14: OK, got < False >
Test 15: OK, got < {?x -> A} >
Test 16: OK, got < {?x -> A, ?z -> C, ?y -> B} >
Test 17: OK, got < {?x -> A} >
Test 18: OK, got < {?x -> ?y} >
Test 19: OK, got < {?y -> ?x} >
Test 20: OK, got < {?z -> 5, ?y -> 5, ?x -> 5} >
Test 21: OK, got < {?z -> <built-in function add>[1,2], ?y -> 4, ?x -> 3} >
>>>  22 / 22 tests successful.
